In [ ]:
#import matplotlib_hangul
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate,KFold
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import optuna
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV, ShapleyImportanceEvaluator
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

In [ ]:
X_train = pd.read_csv('data/feature_enginerring_train.csv', encoding = 'cp949')
X_test = pd.read_csv('data/feature_enginerring_test.csv', encoding = 'cp949')

submission = pd.read_csv('data/sample_submission.csv', encoding = 'cp949')

## __CatBoost__

In [ ]:
numeric_features = ['income_total', 'family_size', 'begin_month', 'age', 'year_of_service', 'child_num','DAYS_BIRTH','DAYS_EMPLOYED',
                    'DAYS_BIRTH_month', 'DAYS_BIRTH_week', 'DAYS_EMPLOYED_month', 'DAYS_EMPLOYED_week',
                    'before_EMPLOYED', 'before_EMPLOYED_month', 'before_EMPLOYED_week','소득/가족','소득/자녀',
                    'BIRTH*id', 'EMP*id', 'BIRTH*EMP', 'possible','id_총DAYS_BIRTH','id_평균DAYS_BIRTH', 'id_최대DAYS_BIRTH', 'id_최소DAYS_BIRTH', 'id_DAYS_BIRTH표준편차', 'id_DAYS_BIRTH변동계수',
                    'id_평균begin_month','id_최대begin_month','id_최소begin_month','id_begin_month표준편차','id_begin_month변동계수',
                    'income_type__총income_total', 'income_type_평균income_total', 'income_type_최대income_total', 'income_type__최소income_total', 'income_type__income_total표준편차', 'income_type__income_total변동계수',
                    'edu_type__총income_total', 'edu_type_평균income_total', 'edu_type_최대income_total', 'edu_type__최소income_total','edu_type__income_total표준편차','edu_type__income_total변동계수',
                    'family_type__총income_total', 'family_type_평균income_total', 'family_type_최대income_total', 'family_type__최소income_total','family_type__income_total표준편차','family_type__income_total변동계수',
                    'house_type__총income_total','house_type_평균income_total','house_type_최대income_total','house_type__최소income_total','house_type__income_total표준편차','house_type__income_total변동계수',
                    'Age_type__총income_total','Age_type_평균income_total','Age_type_최대income_total','Age_type__최소income_total','Age_type__income_total표준편차','Age_type__income_total변동계수',
                    'EMPLOYED_RATIO','income_per_days_birth','income_per_days_birth_X_DAYS_BIRTH','begin_month_X_DAYS_BIRTH',
                    'occyp_type__총income_total','occyp_type_평균income_total','occyp_type_최대income_total','occyp_type__최소income_total','occyp_type__income_total표준편차','occyp_type__income_total변동계수',
                    'income/before_EMPLOYED','income_total_dev','income_total_log','edu_type_num','edu_type_num/DAYS_BIRTH','edu_type_num*income_per_days_birth']
categorical_features= ['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type',
                       'age_group', 'years_group', 'id','work_phone','phone','email','DAYS_BIRTH_class','possible_class',
                       'car_reality','begin_month_class','pos+beg+dBirth']

X_train = X_train[numeric_features+categorical_features]
X_test = X_test[numeric_features+categorical_features]

In [ ]:
X_train.info()

In [ ]:
# num_train = X_train[numeric_features]
# num_test = X_test[numeric_features]

In [ ]:
# # Method: Using SHAP values
# import shap

# # DF, based on which importance is checked
# X_importance = num_test

# # Explain model predictions using shap library:
# model = CatBoostClassifier(verbose=False, random_state=0).fit(num_train, y_train)
# explainer = shap.TreeExplainer(model)
# shap_values = explainer.shap_values(X_importance)

# # Plot summary_plot as barplot:
# shap.summary_plot(shap_values, X_importance, plot_type='bar')


In [ ]:
# shap_sum = np.abs(shap_values).mean(axis=1)[1, :]
# importance_df = pd.DataFrame(
#     [X_importance.columns.tolist(), shap_sum.tolist()]).T
# importance_df.columns = ['column_name', 'shap_importance']
# importance_df = importance_df.sort_values('shap_importance', ascending=False)
# importance_df


In [ ]:
# # feature 중요도가 0 이상
# SHAP_THRESHOLD = 0
# features_selected = importance_df.query('shap_importance > @SHAP_THRESHOLD').column_name.tolist()

In [ ]:
from sklearn.model_selection import StratifiedKFold

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_index, valid_index in skfold.split(X_train, y_train):
    folds.append((train_index, valid_index))

cat_pred = np.zeros((X_train.shape[0], 3))
cat_pred_test = np.zeros((X_test.shape[0], 3))

for fold in range(5):
  print(f'\n training model for CV {fold} \n')
  train_index, valid_index = folds[fold]
  train_x, valid_x = X_train.iloc[train_index], X_train.iloc[valid_index]
  train_y, valid_y = y_train.iloc[train_index], y_train.iloc[valid_index]

  model = CatBoostClassifier(
      cat_features=categorical_features, verbose=False, random_state=0)
  model.fit(train_x, train_y,
            eval_set=[(valid_x, valid_y)],
            early_stopping_rounds=100,
            )

  cat_pred[valid_index] = model.predict_proba(valid_x)
  cat_pred_test += model.predict_proba(X_test) / 5
  print(f'CV Log Loss Score: {log_loss(valid_y, cat_pred[valid_index]):.6f}')

print(f'\tLog Loss: {log_loss(y_train, cat_pred):.6f}')


In [ ]:
def plot_feature_importance(importance, names, model_type):

    feature_importance = np.array(importance)
    feature_names = np.array(names)

    data = {'feature_names': feature_names,
            'feature_importance': feature_importance}
    fi_df = pd.DataFrame(data)

    fi_df.sort_values(by=['feature_importance'], ascending=False, inplace=True)

    plt.figure(figsize=(40, 30))

    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])

    plt.title(model_type + ' Feature Importance')
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature Names')

    plt.savefig('Feature Importance.png', dpi=400)

In [ ]:
plot_feature_importance(model.get_feature_importance(),X_test.columns,'CATBOOST')


In [ ]:
sub = pd.read_csv('data/sample_submission.csv')

In [ ]:
sub.iloc[:, 1:] = cat_pred_test 

In [ ]:
sub.to_csv(f"submission_cat_all_2.csv", index=False)